In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

Read CSV Train and Test

In [ ]:
train_df = pd.read_csv('sign_mnist_train.csv')
test_df = pd.read_csv('sign_mnist_test.csv')

Split Labels and Features

In [ ]:
train_labels_df = train_df.pop('label')
test_labels_df = test_df.pop('label')


Normalize Train and Test Features

In [ ]:
normalized_train_df = train_df / 255.0
normalized_test_df = test_df / 255.0

Convert Pandas Dataframes to NumPy Arrays

In [ ]:
normalized_train_features = normalized_train_df.to_numpy()
normalized_test_features = normalized_test_df.to_numpy()

train_labels = train_labels_df.to_numpy()
test_labels = test_labels_df.to_numpy()

Create and Compile the Model 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(26)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Train and Evaluate Model

In [ ]:
model.fit(normalized_train_features, train_labels, epochs=10)
test_loss, test_accuracy = model.evaluate(normalized_test_features,  test_labels, verbose=2)

Create a Probabilistic Model

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(normalized_test_features)

Visualize Data and Predictions

In [ ]:
class_names = list(string.ascii_uppercase)

def plot_image(i, predictions_array, true_label, image):
  true_label, image = true_label[i], image[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  image = image.reshape((28, 28))
  plt.imshow(image, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)


def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(26))
  plt.yticks([])
  thisplot = plt.bar(range(26), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


rows = 5
columns = 3
total_images = rows * columns
plt.figure(figsize = (2 * 2 * columns, 2 * rows))
for i in range(total_images):
  plt.subplot(rows, 2 * columns, 2 * i + 1)
  plot_image(i, predictions[i], test_labels, normalized_test_features)
  plt.subplot(rows, 2*columns, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()